In [4]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain-community tavily-python


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("TAVILY_API_KEY")

In [7]:
%pip install langsmith

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import os

os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "plan-execute-agent"

In [9]:
from langchain_community.tools.tavily_search import TavilySearchResults

tools = [TavilySearchResults(max_results=3)]

In [10]:
pip install -U langchain-ollama

  Attempting uninstall: langchain-ollama
    Found existing installation: langchain-ollama 0.3.0
    Uninstalling langchain-ollama-0.3.0:
      Successfully uninstalled langchain-ollama-0.3.0
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
from langchain_ollama import ChatOllama
from langchain import hub
from langgraph.prebuilt import create_react_agent

# Choose the LLM that will drive the agent
llm = ChatOllama(model="mistral")
prompt = "You are a helpful assistant."
agent_executor = create_react_agent(llm, tools, prompt=prompt)

In [12]:
agent_executor.invoke({"messages": [("user", "who is the winnner of the us open")]})

{'messages': [HumanMessage(content='who is the winnner of the us open', additional_kwargs={}, response_metadata={}, id='1e83ccad-a08d-45cc-ad0f-0ae52846d8a0'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'mistral', 'created_at': '2025-04-08T05:11:58.289998Z', 'done': True, 'done_reason': 'stop', 'total_duration': 18368158200, 'load_duration': 4843032300, 'prompt_eval_count': 118, 'prompt_eval_duration': 8821439900, 'eval_count': 29, 'eval_duration': 4697729600, 'message': Message(role='assistant', content='', images=None, tool_calls=None), 'model_name': 'mistral'}, id='run-c3641f2b-7f54-469a-b175-4ede50624b5c-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'winner of the us open'}, 'id': 'aca24b1d-a198-49fe-a9aa-51eb672cf898', 'type': 'tool_call'}], usage_metadata={'input_tokens': 118, 'output_tokens': 29, 'total_tokens': 147}),
  ToolMessage(content='[{"title": "Year by Year | History of the US Open - A USTA Event - USOpen.org", "

In [13]:
import operator
from typing import Annotated, List, Tuple
from typing_extensions import TypedDict


class PlanExecute(TypedDict):
    input: str
    plan: List[str]
    past_steps: Annotated[List[Tuple], operator.add]
    response: str

In [14]:
pip show langchain-core


Name: langchain-core
Version: 0.3.51
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: c:\Users\abdul.muhmin\Al&ML\local_ai_agent\LocalAIAgentWithRAG\Restaurant Review AI Agent\venv\Lib\site-packages
Requires: jsonpatch, langsmith, packaging, pydantic, PyYAML, tenacity, typing-extensions
Required-by: langchain, langchain-chroma, langchain-community, langchain-ollama, langchain-text-splitters, langgraph, langgraph-checkpoint, langgraph-prebuilt
Note: you may need to restart the kernel to use updated packages.


In [16]:
from pydantic import BaseModel, Field


class Plan(BaseModel):
    """Plan to follow in future"""

    steps: List[str] = Field(
        description="different steps to follow, should be in sorted order"
    )

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

planner_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """For the given objective, come up with a simple step by step plan. \
This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps. \
The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps.""",
        ),
        ("placeholder", "{messages}"),
    ]
)
planner = planner_prompt | ChatOllama(
    model="mistral", temperature=0
).with_structured_output(Plan)